## 持有时间与仓位算法
1. 读取本地数据
2. 读取信号, 并且修改权重
3. 持有10天的权重分配

## 1_读取本地数据

In [1]:
import pandas as pd
import numpy as np
from jaqs.data import DataView
from jaqs.research import SignalDigger
import warnings

data = pd.read_excel('signal_df.xlsx', index_col='trade_date')

D:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
print(data.tail().stack())

trade_date           
20171218    000001.SZ    0.028519
            000069.SZ    0.044093
            000100.SZ    0.028474
            000402.SZ    0.029721
            000623.SZ    0.037530
            000728.SZ    0.045084
            000776.SZ    0.029339
            000783.SZ    0.037970
            000959.SZ    0.027329
            000963.SZ    0.046964
            002236.SZ    0.106231
            002241.SZ    0.026804
            002310.SZ    0.028444
            002426.SZ    0.026606
            002456.SZ    0.160243
            002475.SZ    0.062133
            002572.SZ    0.064084
            002736.SZ    0.030157
            002831.SZ    0.026710
            002839.SZ    0.045822
            300059.SZ    0.040967
            600016.SH    0.028757
            600018.SH    0.036246
            600048.SH    0.029588
            600068.SH    0.066481
            600297.SH    0.029303
            600340.SH    0.032477
            600369.SH    0.032991
            600376.SH    0

## 2_读取信号, 并且修改权重

In [3]:
import pandas as pd

weight_list = []
for time_index, weight in data.iterrows():
    weiht_result = (weight/weight.sum())
    weight_list.append(weiht_result.to_dict())
stock_df = pd.DataFrame(weight_list, index=data.index)

In [4]:
print(stock_df.tail().stack())

trade_date           
20171218    000001.SZ    0.012165
            000069.SZ    0.018808
            000100.SZ    0.012145
            000402.SZ    0.012678
            000623.SZ    0.016008
            000728.SZ    0.019231
            000776.SZ    0.012515
            000783.SZ    0.016196
            000959.SZ    0.011657
            000963.SZ    0.020032
            002236.SZ    0.045312
            002241.SZ    0.011433
            002310.SZ    0.012133
            002426.SZ    0.011349
            002456.SZ    0.068351
            002475.SZ    0.026503
            002572.SZ    0.027335
            002736.SZ    0.012863
            002831.SZ    0.011393
            002839.SZ    0.019545
            300059.SZ    0.017474
            600016.SH    0.012266
            600018.SH    0.015460
            600048.SH    0.012621
            600068.SH    0.028357
            600297.SH    0.012499
            600340.SH    0.013853
            600369.SH    0.014072
            600376.SH    0

In [5]:
print(stock_df.sum(axis=1))

trade_date
20120105    1.0
20120106    1.0
20120109    1.0
20120110    1.0
20120111    1.0
20120112    1.0
20120113    1.0
20120116    1.0
20120117    1.0
20120118    1.0
20120119    1.0
20120120    1.0
20120130    1.0
20120131    1.0
20120201    1.0
20120202    1.0
20120203    1.0
20120206    1.0
20120207    1.0
20120208    1.0
20120209    1.0
20120210    1.0
20120213    1.0
20120214    1.0
20120215    1.0
20120216    1.0
20120217    1.0
20120220    1.0
20120221    1.0
20120222    1.0
           ... 
20171113    1.0
20171114    1.0
20171115    1.0
20171116    1.0
20171117    1.0
20171120    1.0
20171121    1.0
20171122    1.0
20171123    1.0
20171124    1.0
20171127    1.0
20171128    1.0
20171129    1.0
20171130    1.0
20171201    1.0
20171204    1.0
20171205    1.0
20171206    1.0
20171207    1.0
20171208    1.0
20171211    1.0
20171212    1.0
20171213    1.0
20171214    1.0
20171215    1.0
20171218    1.0
20171219    1.0
20171220    1.0
20171221    1.0
20171222    1.0
Length: 1452,

## 3_持有gap天的权重分配
1. 切片拆分
2. 重塑索引填补权重
3. 将表格求和

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def cal_daily_weight(df, gap=5):
    df = pd.concat([pd.DataFrame(0, list(range(gap-1)), df.columns), df])
    return (pd.rolling_apply(df, gap, sum)/gap).iloc[gap-1:]
    
def change_columns_index(signal):
    new_names = {}
    for c in signal.columns:
        if c.endswith('SZ'):
            new_names[c] = c.replace('SZ', 'XSHE')
        elif c.endswith('SH'):
            new_names[c] = c.replace('SH', 'XSHG')
    signal = signal.rename_axis(new_names, axis=1)
    signal.index = pd.Index(map(lambda x: datetime.strptime(str(x),"%Y%m%d") , signal.index))
    signal.index = pd.Index(map(lambda x: x+timedelta(hours=15) , signal.index))
    return signal

table = change_columns_index(stock_df)
target = cal_daily_weight(table.fillna(0), gap=8)
print(target.head(9).sum(1))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: pd.rolling_apply is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=8,center=False).apply(func=<builtin_function_or_method>,args=<tuple>,kwargs=<dict>)
  import sys


2012-01-05 15:00:00    0.125
2012-01-06 15:00:00    0.250
2012-01-09 15:00:00    0.375
2012-01-10 15:00:00    0.500
2012-01-11 15:00:00    0.625
2012-01-12 15:00:00    0.750
2012-01-13 15:00:00    0.875
2012-01-16 15:00:00    1.000
2012-01-17 15:00:00    1.000
dtype: float64
